# Starbucks Capstone

## Loading the Data

Let's load the provided data into some pandas dataframs and gather some basic information about each

In [ ]:
import pandas as pd
import numpy as np
import os
import io
import math
import json

import matplotlib.pyplot as plt
% matplotlib inline

import boto3
import sagemaker

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [ ]:
# Let's print some information about each our files
portfolio.head()

In [ ]:
profile.head()

In [ ]:
transcript.head()

## Data successfully loaded

We've now got a peek of each of our DataFrames which have been read in. Let's gather some exploratory information about the breakdown for a few of the stats.

In [ ]:
# Plot the income
income_unavailable = sum(pd.isnull(profile['income']))
print('Income reported: ', len(profile) - income_unavailable)
print('Income unreported: ', income_unavailable)

clean_profile = profile.dropna(axis=0)
column_name = 'income'

# Lets see an income breakdown and plot it
ax=plt.subplots(figsize=(6,3))
# get data by column_name and display a histogram
ax = plt.hist(clean_profile[column_name], bins=30)
title=f'Histogram of {column_name} among reporters'
plt.title(title, fontsize=12)
plt.show()
    

In [ ]:
# Plot the income
income_unavailable = len(profile[profile['age'] == 118])
print('age reported: ', len(profile) - income_unavailable)
print('age unreported: ', income_unavailable)

clean_profile = profile.dropna(axis=0)
column_name = 'age'

# Lets see an income breakdown and plot it
ax=plt.subplots(figsize=(6,3))
# get data by column_name and display a histogram
ax = plt.hist(clean_profile[column_name], bins=30)
title=f'Histogram of {column_name} among reporters'
plt.title(title, fontsize=12)
plt.show()

In [ ]:
# Let's see our gender breakdown
print('Total: ', len(profile))
print('Women: ', len(profile[profile['gender'] == 'F']))     
print('Men: ', len(profile[profile['gender'] == 'M']))
print('Other: ', len(profile[profile['gender'] == 'O']))
print('None: ', sum(pd.isnull(profile['gender'])))

In [ ]:
# Let's see what type events are available
types = transcript.event.unique()
for event in types:
    print(event, '    \t:\t', len(transcript[transcript['event'] == event]))     

### Thinking about how to proceed with data pre-processing

At this point, we're ready to start transforming our data in order to maximize the amount of usefulness we'll gain from performing the Principal Component Analysis.

Something we want to be able to continue to referenece is the need for our data to be kept within terms of each customer. In order to do that, we'll have to make some modifications to the profile dataFrame and include various statistics derived from the other data.

In [ ]:
def user_stats_df(df):
    # make a new copy of the profile dataframe
    new_df = df
    types = transcript.event.unique()         
    
    event_count_map = { 'offer received': [],
                        'offer viewed': [],
                        'transaction': [],
                        'offer completed': [] }
    
    # Let's take a count of each user's records for each event type
    for index, row in new_df.iterrows():    
        pid = row['id']
        user_events = transcript[transcript['person'] == pid]
    
        for event in types:
            # Add the new column with the calculated values for each event type
            event_count_map[event].append(len(user_events[user_events['event'] == event]))
    
    # Now add each column based on the results above
    new_df['received'] = event_count_map['offer received']
    new_df['viewed'] = event_count_map['offer viewed']
    new_df['transactions'] = event_count_map['transaction']
    new_df['completed'] = event_count_map['offer completed']
    
    return new_df

In [ ]:
%%time

result = user_stats_df(profile)

In [ ]:
print(len(result))
result.head(10)

In [ ]:
## TODO

# 1) Convert membership date to age

# 2) need an offer df? columns: id, person, number_of views, initial_time_to_view, time_to_complete
#    avg_response_time add a column for avg offer age when viewed (time(viewed) - time(recieved))
#    avg_completion_time for avg offer age when completed (time(completed) - time(viewed))


# Drop a few columns
We can drop a couple things. Firstly, we no longer need the id as it cannot be normalized and we no longer have a use for it. Secondly, for our early model, lets drop any users who haven't reported their age - this is manifested by a reported age of '118'

In [ ]:
df = profile[profile['age'] != 118]
print(f'Size after dropping unreported age: {len(df)}')

# TODO Turn gender into a 0123 class so it doesnt need to be dropped
clean_df = df.drop(['id','gender'], 1)
print('Dropped id column')
clean_df.head()

## Normalize the data we've got

In [ ]:
# scale numerical features into a normalized range, 0-1

from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
# store them in this dataframe
df_scaled=pd.DataFrame(scaler.fit_transform(clean_df.astype(float)))

# get same features and profile indices
df_scaled.columns=clean_df.columns
df_scaled.index=clean_df.index

df_scaled.head()

In [ ]:
df_scaled.describe()

In [ ]:
from sagemaker import get_execution_role

session = sagemaker.Session() # store the current SageMaker session

# get IAM role
role = get_execution_role()
print(role)

In [ ]:
# get default bucket
bucket_name = session.default_bucket()
print(bucket_name)
print()

In [ ]:
# define location to store model artifacts
prefix = 'customer_profiles'

output_path='s3://{}/{}/'.format(bucket_name, prefix)

print('Training artifacts will be uploaded to: {}'.format(output_path))

In [ ]:
# define a PCA model
from sagemaker import PCA

# this is current features - 1
# you'll select only a portion of these to use, later
N_COMPONENTS=6

pca_SM = PCA(role=role,
             train_instance_count=1,
             train_instance_type='ml.c4.xlarge',
             output_path=output_path, # specified, above
             num_components=N_COMPONENTS, 
             sagemaker_session=session)


In [ ]:
# convert df to np array
train_data_np = df_scaled.values.astype('float32')

# convert to RecordSet format
formatted_train_data = pca_SM.record_set(train_data_np)

In [ ]:
%%time

# train the PCA mode on the formatted data
pca_SM.fit(formatted_train_data)

In [ ]:
training_job_name = 'pca-2020-01-07-10-47-05-733' ## NEEDS TO BE UPDATED MANUALLY

# where the model is saved, by default
model_key = os.path.join(prefix, training_job_name, 'output/model.tar.gz')
print(model_key)

# download and unzip model
boto3.resource('s3').Bucket(bucket_name).download_file(model_key, 'model.tar.gz')

# unzipping as model_algo-1
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

In [ ]:
import mxnet as mx

# loading the unzipped artifacts
pca_model_params = mx.ndarray.load('model_algo-1')

# what are the params
print(pca_model_params)

In [ ]:
# get selected params
s=pd.DataFrame(pca_model_params['s'].asnumpy())
v=pd.DataFrame(pca_model_params['v'].asnumpy())

In [ ]:
# looking at top 5 components
n_principal_components = 5

start_idx = N_COMPONENTS - n_principal_components  # 33-n

# print a selection of s
print(s.iloc[start_idx:, :])

In [ ]:
# Calculate the explained variance for the top n principal components
# you may assume you have access to the global var N_COMPONENTS
def explained_variance(s, n_top_components):
    '''Calculates the approx. data variance that n_top_components captures.
       :param s: A dataframe of singular values for top components; 
           the top value is in the last row.
       :param n_top_components: An integer, the number of top components to use.
       :return: The expected data variance covered by the n_top_components.'''
    
    start_idx = N_COMPONENTS - n_top_components  ## 33-3 = 30, for example
    # calculate approx variance
    exp_variance = np.square(s.iloc[start_idx:,:]).sum()/np.square(s).sum()
    
    return exp_variance[0]

In [ ]:
# test cell
n_top_components = 4 # select a value for the number of top components

# calculate the explained variance
exp_variance = explained_variance(s, n_top_components)
print('Explained variance: ', exp_variance)

In [ ]:
import seaborn as sns

def display_component(v, features_list, component_num, n_weights=10):
    
    # get index of component (last row - component_num)
    row_idx = N_COMPONENTS-component_num

    # get the list of weights from a row in v, dataframe
    v_1_row = v.iloc[:, row_idx]
    v_1 = np.squeeze(v_1_row.values)

    # match weights to features in profile dataframe, using list comporehension
    comps = pd.DataFrame(list(zip(v_1, features_list)), 
                         columns=['weights', 'features'])

    # we'll want to sort by the largest n_weights
    # weights can be neg/pos and we'll sort by magnitude
    comps['abs_weights']=comps['weights'].apply(lambda x: np.abs(x))
    sorted_weight_data = comps.sort_values('abs_weights', ascending=False).head(n_weights)

    # display using seaborn
    ax=plt.subplots(figsize=(10,6))
    ax=sns.barplot(data=sorted_weight_data, 
                   x="weights", 
                   y="features", 
                   palette="Blues_d")
    ax.set_title("PCA Component Makeup, Component #" + str(component_num))
    plt.show()

In [ ]:
# display makeup of first component
num=2
display_component(v, df_scaled.columns.values, component_num=num, n_weights=10)

In [ ]:
%%time

## DEPLOY THE PCA MODEL

# this takes a little while, around 7mins
pca_predictor = pca_SM.deploy(initial_instance_count=1, 
                              instance_type='ml.t2.medium')

In [ ]:
# pass np train data to the PCA model
train_pca = pca_predictor.predict(train_data_np)

In [ ]:
# check out the first item in the produced training features
data_idx = 0
print(train_pca[data_idx])

In [ ]:
# create dimensionality-reduced data
def create_transformed_df(train_pca, profiles_scaled, n_top_components):
    ''' Return a dataframe of data points with component features. 
        The dataframe should be indexed by User Profile and contain component values.
        :param train_pca: A list of pca training data, returned by a PCA model.
        :param profiles_scaled: A dataframe of normalized, original features.
        :param n_top_components: An integer, the number of top components to use.
        :return: A dataframe, indexed by Profile, with n_top_component values as columns.        
     '''
    # create new dataframe to add data to
    profiles_transformed=pd.DataFrame()

    # for each of our new, transformed data points
    # append the component values to the dataframe
    for data in train_pca:
        # get component values for each data point
        components=data.label['projection'].float32_tensor.values
        profiles_transformed=profiles_transformed.append([list(components)])

    # index by profile, just like profiles_scaled
    profiles_transformed.index=profiles_scaled.index

    # keep only the top n components
    start_idx = N_COMPONENTS - n_top_components
    profiles_transformed = profiles_transformed.iloc[:,start_idx:]
    
    # reverse columns, component order     
    return profiles_transformed.iloc[:, ::-1]

In [ ]:
# specify top n
top_n = 4

# call your function and create a new dataframe
pf_transformed = create_transformed_df(train_pca, df_scaled, n_top_components=top_n)

# add descriptive columns
PCA_list=[]

for i in range(top_n):
    PCA_list.append(f'c_{i}')

pf_transformed.columns=PCA_list 

# print result
pf_transformed.head()

In [ ]:
# delete predictor endpoint
session.delete_endpoint(pca_predictor.endpoint)

In [ ]:
### K-MEANS model

# define a KMeans estimator
from sagemaker import KMeans

NUM_CLUSTERS = 6

kmeans = KMeans(role=role,
                train_instance_count=1,
                train_instance_type='ml.c4.xlarge',
                output_path=output_path, # using the same output path as was defined, earlier              
                k=NUM_CLUSTERS)


In [ ]:
# convert the transformed dataframe into record_set data
kmeans_train_data_np = pf_transformed.values.astype('float32')
kmeans_formatted_data = kmeans.record_set(kmeans_train_data_np)

In [ ]:
%%time
# train kmeans
kmeans.fit(kmeans_formatted_data)

In [ ]:
%%time
# deploy the model to create a predictor
kmeans_predictor = kmeans.deploy(initial_instance_count=1, 
                                 instance_type='ml.t2.medium')

In [ ]:
# get the predicted clusters for all the kmeans training data
cluster_info=kmeans_predictor.predict(kmeans_train_data_np)

In [ ]:
# print cluster info for first data point
data_idx = 0

print('user is: ', pf_transformed.index[data_idx])
print()
print(cluster_info[data_idx])

In [ ]:
# get all cluster labels
cluster_labels = [c.label['closest_cluster'].float32_tensor.values[0] for c in cluster_info]

In [ ]:
# count up the points in each cluster
cluster_df = pd.DataFrame(cluster_labels)[0].value_counts()

print(cluster_df)

In [ ]:
# another method of visualizing the distribution
# display a histogram of cluster counts
ax =plt.subplots(figsize=(6,3))
ax = plt.hist(cluster_labels, bins=8,  range=(-0.5, 7.5), color='blue', rwidth=0.5)

title="Histogram of Cluster Counts"
plt.title(title, fontsize=12)
plt.show()

In [ ]:
# delete kmeans endpoint
session.delete_endpoint(kmeans_predictor.endpoint)

In [ ]:
#can be found under 'training jobs' in sagemaker menu
kmeans_job_name = 'kmeans-2020-01-07-11-01-18-320'

model_key = os.path.join(prefix, kmeans_job_name, 'output/model.tar.gz')

# download the model file
boto3.resource('s3').Bucket(bucket_name).download_file(model_key, 'model.tar.gz')
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

In [ ]:
# get the trained kmeans params using mxnet
kmeans_model_params = mx.ndarray.load('model_algo-1')

print(kmeans_model_params)

In [ ]:
# get all the centroids
cluster_centroids=pd.DataFrame(kmeans_model_params[0].asnumpy())
cluster_centroids.columns=pf_transformed.columns

display(cluster_centroids)

In [ ]:
# generate a heatmap in component space, using the seaborn library
plt.figure(figsize = (12,9))
ax = sns.heatmap(cluster_centroids.T, cmap = 'YlGnBu')
ax.set_xlabel("Cluster")
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
ax.set_title("Attribute Value by Centroid")
plt.show()

In [ ]:
# what do each of these components mean again?
# let's use the display function, from above
component_num=3
display_component(v, df_scaled.columns.values, component_num=component_num)

In [ ]:
# add a 'labels' column to the dataframe
pf_transformed['labels']=list(map(int, cluster_labels))

# sort by cluster label 0-6
sorted_profiles = pf_transformed.sort_values('labels', ascending=True)
# view some pts in cluster 0
sorted_profiles.head(20)

In [ ]:
# get all profiles with label == 1
cluster=pf_transformed[pf_transformed['labels']==2]
cluster.head()